In [1]:
#Install libraries - remove the comment to do this

!pip install langchain
!pip install langchain-community
!pip install openai
!pip install PyPDF2
!pip install tiktoken
!pip install faiss-cpu

ERROR: Could not find a version that satisfies the requirement faiss-spu (from versions: none)
ERROR: No matching distribution found for faiss-spu


  Using cached faiss_cpu-1.10.0-cp312-cp312-win_amd64.whl.metadata (4.5 kB)
Using cached faiss_cpu-1.10.0-cp312-cp312-win_amd64.whl (13.7 MB)


In [14]:
#Import classes from libraries

from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI

In [ ]:
#The "os" module allows you to interact with the os and perform varioous os-related tasks

import os

os.environ["OPENAI_API_KEY"] = 

# Simple LLM call with generic knowledge


In [16]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

In [17]:
our_query = "Who is nicole colgan"
print(llm.predict(our_query))

There is limited information available about Nicole Colgan. It is possible that she is a private individual or does not have a significant online presence. Without further context, it is difficult to provide more information about her.


# LLM call with our pdf as a reference

In [6]:
data = PdfReader('C:/Users/colgann/Training/Langchain/Nicole.pdf')

In [18]:
combined_text = ''
#Enumerate allows you to iterate over an iteratable (like list of pages) while keeping track of each item in the list and its index
for i, page in enumerate(data.pages):
    text = page.extract_text()
    print(i, page, text)
    if text:
        combined_text += text

0 {'/Type': '/Page', '/Parent': IndirectObject(2, 0, 3081841311088), '/Resources': {'/Font': {'/F1': IndirectObject(5, 0, 3081841311088), '/F2': IndirectObject(12, 0, 3081841311088)}, '/ExtGState': {'/GS10': IndirectObject(10, 0, 3081841311088), '/GS11': IndirectObject(11, 0, 3081841311088)}, '/ProcSet': ['/PDF', '/Text', '/ImageB', '/ImageC', '/ImageI']}, '/MediaBox': [0, 0, 595.32, 841.92], '/Contents': IndirectObject(4, 0, 3081841311088), '/Group': {'/Type': '/Group', '/S': '/Transparency', '/CS': '/DeviceRGB'}, '/Tabs': '/S', '/StructParents': 0} Nicole Colgan is a softw are dev from Galway  
Nicole Colgan is a business owner and personal trainer  
Nicole Colgan specialises in ai develeopment  


In [19]:
combined_text

'Nicole Colgan is a softw are dev from Galway  \nNicole Colgan is a business owner and personal trainer  \nNicole Colgan specialises in ai develeopment  '

## Lets try add to model with our own data

In [20]:
#Need to split text - in our example, its small ammount so not really neccesary but it will be in production scenarios
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 200,
    chunk_overlap = 20,
    length_function = len
)
finalData = text_splitter.split_text(combined_text)

In [21]:
len(finalData)
finalData

['Nicole Colgan is a softw are dev from Galway  \nNicole Colgan is a business owner and personal trainer  \nNicole Colgan specialises in ai develeopment']


Create vector embedding for text
This vector is used to store and retrieve vector embeddings for text efficiently

FAISS stands for 'Facebook AI Similarity Search" whic is an open-source library develped by Facebooks AI research
FAISS helps you to search for similar items within large datasets quickly - it works with embeddings or other types of data that can be represented as vectors

In [22]:
# Such a small ammoun of data we only get one chunk
from langchain.vectorstores import FAISS

embeddings = OpenAIEmbeddings()
# from_texts - create search index from text that FAISS can work with

documentSearch = FAISS.from_texts(finalData, embeddings)

### Load a chain for question answering (QA) tasks
A chain in Langchain is a way to connect different components (Like llms, retrievers, memory)
eg Instead of calling an LLM directly, a chain allows you to define a sequence of operations like
- Take user input
- retrieve relevant documents
- Process them with an llm
- Return a response

### A QA chain is a type of chain designed to:
- Take a user question
- retrieve relevant docs (if using a retriever)
- Pass them to an llm
- Generate a response using he context from those docs

There are multiple chain types eg map reduce, refine etc

In [26]:
# Convert FAISS into a retriever (retrieve relevant chunks)
from langchain.chains import RetrievalQA
retriever = documentSearch.as_retriever()

#Create retrievalQA chain which connects lllm to a retriever (FAISS) for document-based question answering
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever
    )

# Ask a question
response = qa_chain.run("What is nicoles colgans job")
print(response)

C:\Users\colgann\AppData\Local\Temp\ipykernel_30528\2422802596.py:12: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run("What is nicoles colgans job")


Nicole Colgan is a software developer specializing in AI development.
